In [1]:
%load_ext autoreload
%autoreload 2
%matplotlib inline
%config InlineBackend.figure_format = 'retina'

# import os
# os.environ["TF_GPU_ALLOCATOR"]="cuda_malloc_async" # this seems to be highly important for totally utilizing your GPU's memory, but it also breaks the profiler's memory breakdown
# note that GradientTape needs several times the memory needed to compute the fidelity of a single circuit

import numpy as np
import qutip as qt
from QOGS.optimizer.tf_adam_optimizer import AdamOptimizer
from QOGS.gate_sets import PI_GRAPE, GRAPE
import matplotlib.pyplot as plt


Need tf version 2.3.0 or later. Using tensorflow version: 2.8.0



In [2]:
q_dim = 3
c_dim = 10
DAC_time_resolution = 2 # in ns

# In GHz = cycles / ns
anharm = -.180
kerr = -1e-5*0
chi = -0.25e-3
chi_prime = 0
drive = D = 2 * np.pi * 1e-3

a = qt.tensor(qt.destroy(c_dim), qt.qeye(q_dim))
b = qt.tensor(qt.qeye(c_dim), qt.destroy(q_dim))
gf = qt.tensor(qt.qeye(c_dim), qt.basis(q_dim, 0) * qt.basis(q_dim, 2).dag())
fg = gf.dag()
# hh = qt.tensor(qt.qeye(c_dim), qt.basis(q_dim, 3) * qt.basis(q_dim, 3).dag())
ad = a.dag()
bd = b.dag()
# H0 = (anharm/2) * bd * bd * b * b
# H0 = (3 * anharm) * hh
H0 = (chi) * ad * a * bd * b
H0 += (kerr/2) * ad * ad * a * a
H0 += (chi_prime / 6) * ad * ad * a * a * bd * b
H0 *= 2*np.pi
Hcs = [D*(gf + fg), 1j*D*(gf - fg)]

init_states = [
    ((qt.tensor(qt.basis(c_dim, 0), qt.basis(q_dim, 0)) 
        + qt.tensor(qt.basis(c_dim, 4), qt.basis(q_dim, 0))) / np.sqrt(2)
        + qt.tensor(qt.basis(c_dim, 2), qt.basis(q_dim, 0))).unit()
]

final_states = [
    ((qt.tensor(qt.basis(c_dim, 0), qt.basis(q_dim, 2)) 
        + qt.tensor(qt.basis(c_dim, 4), qt.basis(q_dim, 2))) / np.sqrt(2)
        - qt.tensor(qt.basis(c_dim, 2), qt.basis(q_dim, 2))).unit()
]

success_op = qt.tensor(qt.qeye(c_dim), qt.fock_dm(q_dim, 2))
# jump_op = qt.tensor(qt.qeye(c_dim), qt.fock_dm(q_dim, 0))
jump_op = qt.tensor(qt.qeye(c_dim), qt.fock_dm(q_dim, 0)) / 2 - qt.tensor(qt.qeye(c_dim), qt.fock_dm(q_dim, 2)) / 2

In [3]:
PI = True
synth_params = {
    'N_blocks': 601, # note that the length of the pulse is this times the DAC_time_resolution
    'N_multistart' : 16, #Batch size (number of circuit optimizations to run in parallel)
    'epochs' : 3000, #number of epochs before termination
    'epoch_size' : 5, #number of adam steps per epoch
    'learning_rate' : 0.1, #adam learning rate
    'term_fid' : 0.99, #0.995, #terminal fidelitiy
    'dfid_stop' : 1e-5/2, #stop if dfid between two epochs is smaller than this number
    'initial_states' : init_states, #qubit tensor oscillator, start in |g> |0>
    'target_states' : final_states, #end in |e> |target>.
    'name' : 'PI GRAPE SNAP binom Z' if PI else 'GRAPE SNAP binom Z', #name for printing and saving
    'use_phase' : True,
    'filename' : None, #if no filename specified, results will be saved in this folder under 'name.h5'
}

# We initialize the gateset here
gate_set_params = {
    'H_static' : H0,
    'H_control' : Hcs,
    'DAC_delta_t' : DAC_time_resolution,
    'inplace' : False, # true uses less memory, but is slower. Just use false
    'scale' : 1.0, # range of DAC amplitudes for initial random waves
    'bandwidth' : 0.1,
    'ringup' : 40,
    'jump_ops' : jump_op,
    'jump_weights' : synth_params["N_blocks"] * DAC_time_resolution / 50000 * 20, 
    'success_op' : success_op,
    'success_weight' : 1.0*0,
    'gatesynthargs': synth_params
}



if PI: GRAPE_gate_set = PI_GRAPE(**gate_set_params)
else: GRAPE_gate_set = GRAPE(**gate_set_params)
gate_set_params["jump_weights"]

2022-06-13 03:23:09.320044: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:952] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2022-06-13 03:23:09.347012: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:952] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2022-06-13 03:23:09.347205: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:952] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2022-06-13 03:23:09.348137: I tensorflow/core/platform/cpu_feature_guard.cc:152] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  SSE3 SSE4.1 SSE4.2 AVX
To enable them in other operations, rebuild TensorFlow with the appropriate 

0.4808

In [4]:
#create optimization object. 
#initial params will be randomized upon creation
opt = AdamOptimizer(GRAPE_gate_set)

#print optimization info. this lives in gatesynth, since we eventually want to fully abstract away the optimizer
GRAPE_gate_set.best_fidelity()

2022-06-13 03:23:11.538290: I tensorflow/compiler/xla/service/service.cc:171] XLA service 0x7fda302d8750 initialized for platform CUDA (this does not guarantee that XLA will be used). Devices:
2022-06-13 03:23:11.538325: I tensorflow/compiler/xla/service/service.cc:179]   StreamExecutor device (0): NVIDIA GeForce RTX 3080, Compute Capability 8.6
2022-06-13 03:23:11.547455: I tensorflow/compiler/mlir/tensorflow/utils/dump_mlir_util.cc:237] disabling MLIR crash reproducer, set env var `MLIR_CRASH_REPRODUCER_DIRECTORY` to enable.
2022-06-13 03:23:12.676417: I tensorflow/compiler/jit/xla_compilation_cache.cc:402] Compiled cluster using XLA!  This line is logged at most once for the lifetime of the process.
2022-06-13 03:23:12.805651: I tensorflow/core/util/cuda_solvers.cc:179] Creating GpuSolver handles for stream 0x7ccd870


0.00010426571

In [5]:
#run optimizer.
#note the optimizer can be stopped at any time by interrupting the python consle,
#and the optimization results will still be saved and part of the opt object.
#This allows you to stop the optimization whenever you want and still use the result.
# Note that you will not want to use the performance profiler while using 'inplace' mode. You will run out of memory
opt.optimize()#logdir='logs')

Start time: 2022-06-13 03:23:13
 Epoch: 23 / 3000 Max Fid: 0.179491 Avg Fid: 0.173362 Max dFid: 0.015791 Avg dFid: 0.015556 Elapsed time: 0:02:28.440509 Expected remaining time: 5:20:13.364961

In [ ]:
GRAPE_gate_set.print_info()

In [ ]:
# Load the TensorBoard notebook extension.
%load_ext tensorboard

In [ ]:
# Launch TensorBoard and navigate to the Profile tab to view performance profile
%tensorboard --logdir=logs